In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
transforms = torchvision.transforms
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import gensim
from gensim import corpora
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.layers import Embedding
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

nltk.download(['punkt','wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
# downloading dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')

In [27]:
print(df.columns)
df.describe()

Index(['review', 'sentiment'], dtype='object')


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


In [28]:
zero_1 = pd.get_dummies(df['sentiment'])
df = df.drop('sentiment', axis=1) 
df = df.join(zero_1)
df.head()

,review,negative,positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production. <br /><br />The...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically there's a family where a little boy ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1


In [29]:
def text_processing(text):
  words=word_tokenize(text)

  words_without_punc=[]
  for w in words:
    if w.isalpha():
      words_without_punc.append(w.lower())

  clean_words=[]
  stop_words = set(stopwords.words("english"))
  for w in words_without_punc:
    if w not in stop_words:
      clean_words.append(w)
    
  lemmatized_words=[]
  for w in clean_words:
    lemmatized_words.append(WordNetLemmatizer().lemmatize(w))
  return ' '.join(lemmatized_words)

In [30]:
df['review'] = df['review'].apply(text_processing)
df.to_csv('/content/drive/MyDrive/NLPlay_with_Transformers/pre_processed_data.csv', index=False)
df.head()

,review,negative,positive
0,one reviewer mentioned watching oz episode hoo...,0,1
1,wonderful little production br br filming tech...,0,1
2,thought wonderful way spend time hot summer we...,0,1
3,basically family little boy jake think zombie ...,1,0
4,petter mattei love time money visually stunnin...,0,1


In [31]:
X = df.iloc[:, 0].values
Y = df.iloc[:, 2].values

In [32]:
max_words = 10000
max_len = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=max_len)
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=60, test_size=0.20)
print(len(X_train))
print(len(X_test))

40000
10000


In [33]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_gru.h5',
    save_weights_only=False,
    mode='max',
    monitor='val_accuracy',
    save_best_only=True)

In [34]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 64))
model1.add(layers.SimpleRNN(10,dropout=0.5))
model1.add(layers.Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test),verbose=1,callbacks = [model_checkpoint_callback])

Epoch 1/10
313/313 [==============================] - 108s 262ms/step - loss: 0.6418 - accuracy: 0.6150 - val_loss: 0.4534 - val_accuracy: 0.8098
Epoch 2/10
313/313 [==============================] - 82s 263ms/step - loss: 0.3915 - accuracy: 0.8404 - val_loss: 0.3734 - val_accuracy: 0.8511
Epoch 3/10
313/313 [==============================] - 83s 266ms/step - loss: 0.3154 - accuracy: 0.8754 - val_loss: 0.3871 - val_accuracy: 0.8439
Epoch 4/10
313/313 [==============================] - 83s 265ms/step - loss: 0.2531 - accuracy: 0.9031 - val_loss: 0.3567 - val_accuracy: 0.8638
Epoch 5/10
313/313 [==============================] - 82s 263ms/step - loss: 0.1991 - accuracy: 0.9276 - val_loss: 0.3699 - val_accuracy: 0.8599
Epoch 6/10
313/313 [==============================] - 82s 263ms/step - loss: 0.1520 - accuracy: 0.9502 - val_loss: 0.3888 - val_accuracy: 0.8567
Epoch 7/10
313/313 [==============================] - 82s 262ms/step - loss: 0.1298 - accuracy: 0.9574 - val_loss: 0.4139 - val_a

In [35]:
model = tf.keras.models.load_model('model_gru.h5')
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8651000261306763
